# FINAL PROJECT 

## Final Exam Instructions

### Write a Python program that takes as input a file containing DNA sequences in multi-FASTA format, and computes the answers to the following questions. 


### 1. How many records are in the file? A record in a FASTA file is defined as a single-line header, followed by lines of sequence data. The header line is distinguished from the sequence data by a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier of the sequence, and the rest of the line is an optional description of the entry. There should be no space between the ">" and the first letter of the identifier. 

In [1]:
# Import Biopython and the SeqIO module
import Bio
from Bio import SeqIO

In [2]:
# Open the fasta file containing the sequences to be analysed
fasta = open("dna2.fasta")

In [3]:
# Each sequence starts with a first line that always has the major symbol first, 
# which allows us to identify the number of different sequences there are
num_records = 0
for line in fasta:
    if line.startswith(">"):
        num_records +=1

In [4]:
num_records

18

###  2. What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence? Is there more than one longest or shortest sequence? What are their identifiers? 

In [5]:
# Through the parse method included in the SeqIO module, 
# we will be able to extract the different parts of a fasta file
fasta_file = SeqIO.parse("dna2.fasta","fasta")

In [6]:
list_records = []
list_seq = []
list_ids = []
long_records = []

for seq_record in fasta_file:
    list_records.append(seq_record)
    long_records.append(len(seq_record))
    list_ids.append(seq_record.id)
    record_str = str(seq_record.seq)
    list_seq.append(record_str)
    print(repr(seq_record.seq))
    print(len(seq_record))

Seq('CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGG...GCC')
4635
Seq('CGCGATCCAGTAGCGCTTGTAGCCGAGCGCTTCGGCACGCTTCGCGAGCGCGAT...GCC')
1151
Seq('CTCGACGCGCTCCGCGTCGAGGTCGCCCGACGTCTCGCGCAGCAACTGATTCAA...TCG')
4894
Seq('CGTGCTCGGCACGACTATCAGCCCGTATCTGTTTTTCTGGCAGGCCTCCCAGGA...CCG')
3511
Seq('TGCATCGGCGACGGCTACTGGTTCCTCGAGAAAGCGCGCAACTACGCGAGCGAG...TCA')
4076
Seq('GCCCGTGCCGACGCGCCGCATCGTCAGCGCGCGGCGCTCGTGGTCCTGCCACGC...CGA')
2867
Seq('TGCGGGCGGCCTGTGGGCGGTCGGCGCCGGGCTGTCGCAGCCGCTGTTCCACGG...GCG')
442
Seq('GGACCGGCGAGGAAGTGGTCGGCGTCGAACAGGACGATCGCTGCGTCACGTTGC...CTC')
890
Seq('GCCGCGGGCGCCGGACGGAGCACGCGCATGATCCCGCTTTCCCATCCGTCCTTC...TCG')
967
Seq('ACTGCCGCGTTCAGCGCGAGAATGTTCGTCTGGAACGCAATCCCTTCGATTACC...GAA')
4338
Seq('TATAGGCCTGGACCTGCGTGCGGGCGAGATCGGCGTCTTCGCCGAAGGTGCCGT...GAG')
1352
Seq('TCGCGGGCCTCGCGCTGCCGTTCGCGGCGCTCGCGTTGCTGGAGGTGGTCGTGC...CAG')
4564
Seq('GTCGATCGACACGACGCTCGCGCAGCGCGACGCGAAGGCCGCGTGAGCGCACGA...CGT')
4804
Seq('CGCAGGTGGCAGCATTTAAGATCTCGCAAGGTGGTGTCTCCCCCATCAT

In [7]:
max(long_records)

4894

In [8]:
min(long_records)

115

In [9]:
long_records.index(min(long_records))

16

In [10]:
# This way you know the identifiers of the longest and shortest sequence
list_ids[2]

'gi|142022655|gb|EQ086233.1|255'

In [11]:
list_ids[16]

'gi|142022655|gb|EQ086233.1|346'

### 3. In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5'). For instance, the three possible forward reading frames for the sequence AGGTGACACCGCAAGCCTTATATTAGC are: 

### AGG TGA CAC CGC AAG CCT TAT ATT AGC

### A GGT GAC ACC GCA AGC CTT ATA TTA GC

### AG GTG ACA CCG CAA GCC TTA TAT TAG C 

### These are called reading frames 1, 2, and 3 respectively. An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.

### Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to identify all ORFs present in each sequence of the FASTA file, and answer the following questions: what is the length of the longest ORF in the file? What is the identifier of the sequence containing the longest ORF? For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? What is the starting position of the longest ORF in the sequence that contains it? 

In [12]:
# The open reading frames are called ORFs, and they always start with ATG and end with TAA/TAG/TGA.
# This function searches for all the forward ORFs in all the sequences we have, 
# and will update the value of max_long_ORF every time the value is greater than the one it has
# The reading_frame parameter with values of 0, 1 or 2, allows the three types of readings to be performed

def find_orf(seq, reading_frame):
    max_long_ORF = 0
    pos_max_ORF = 0
    codon_start = "ATG"
    codon_stop = ["TAA","TAG","TGA"]

    for i in range(reading_frame,len(seq)-6,3):
        if seq[i:i+3] == codon_start:
            for j in range(i+3,len(seq)-3,3):
                if seq[j:j+3] in codon_stop: 
                    len_ORF = j+3-i
                    if max_long_ORF < len_ORF:
                        max_long_ORF = len_ORF
                        pos_max_ORF = i
                    break
                    
    return max_long_ORF, pos_max_ORF

In [13]:
# In the case of the three reads, in the last position we have the length 2394 which is the longest ORF, 
# with the first position corresponding to the A of ATG at 384, and which is sequence number 3

list_orfs = []
for seq in list_seq:
    max_long, max_pos = find_orf(seq,0)
    list_orfs.append((max_long,max_pos,list_seq.index(seq)))
sorted(list_orfs)

[(0, 0, 6),
 (0, 0, 16),
 (42, 27, 8),
 (90, 159, 13),
 (105, 858, 1),
 (120, 81, 7),
 (180, 819, 10),
 (195, 1224, 17),
 (204, 597, 15),
 (249, 444, 14),
 (312, 1389, 9),
 (1044, 2337, 11),
 (1059, 528, 4),
 (1296, 978, 0),
 (1443, 291, 2),
 (1509, 1527, 12),
 (1560, 561, 5),
 (2394, 384, 3)]

In [14]:
# Knowing that it is sequence number 3, it is possible to find the identifier for it

list_records[3]

SeqRecord(seq=Seq('CGTGCTCGGCACGACTATCAGCCCGTATCTGTTTTTCTGGCAGGCCTCCCAGGA...CCG'), id='gi|142022655|gb|EQ086233.1|45', name='gi|142022655|gb|EQ086233.1|45', description='gi|142022655|gb|EQ086233.1|45 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[])

### 4. A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.

In [15]:
seq = list_seq[0]

In [16]:
seq

'CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGGGAGGATCTCGGCGGCGCCAACTATGCGGTCTTTCGGCTCGAAAGCCAGTTCCAGACCTCCGACGGCGCGCTGACCGTGCCCGGCTCCGCATTCAGTTCGCAAGCCTACGTCGGGCTCGGCGGCGACTGGGGGACCGTGACGCTCGGGCGCCAGTTCGATTTCGTCGGCGATCTGATGCCGGCTTTCGCGATCGGCGCGAACACGCCGGCCGGCCTGCTCGCGTGGGGCTTGCCGGCGAATGCGTCGGCGGGCGGTGCGCTCGACAACCGCGTGTGGGGCGTCCAGGTGAACAATGCGGTGAAGTACGTGAGCCCGACGTTCGGCGGATTGTCGTTCGGCGGCCTGTGGGGCTTCGGCAACGTGCCCGGCACGGTCGCGCGCAGCAGCGTGCAAAGCGCGATGCTGTCCTACACGCAAGGCGCGTTCAGCGCCGCGCTCGCTTATTTCGGCCAGCACGATGTAACTGCCGGTGGCAATCTGCGCAATTTCTCGGGCGGTGCAGGCTACAACGTCGGGCAGTTCCGCGTCTTCGGCATGGTGTCGGACGTGCGGATCAGCGCCGCCGCGCCGCTGCGGGCCACGACCTATGACGGCGGCTTGACCTATGCGGTCACGCCGGCGTTGCAGCTCGGCGGCGGCTTCCAGTACCAGCAGCGCGGCGGCGACATCGGCTCGGCCAACCAGGTCACGTTGAGCGCCGACTATTCGCTGTCGAAGCGTACCGGCCTTTACGTGGTATTCGCACGCGGGCACGACAGTGCGTATGGCGCGCAGGTCGAGGCGGCGCTCGGCGGGGCGGCGTCCGGCTCGACGCAGACCGCGGTCCGGCTCGGGCTGCGGCATCAGTTCTGACGATGCGCGAGAAACACGGGCTGCCGCGTACGCCGCGCGCGAGCCCGTGTTTTTCCGCCGGATTCAGAACCGATGCATCATCCCGACGCGCAA

In [17]:
## A function is created that for a given length n,
# returns the existing repetitions of that length and the number of times it is found.

def count_repeat_elements(seq, length):
    subseq_n = []

    for i in range(len(seq)-length):
        subseq_n.append(seq[i:i+length])
              
    return subseq_n

In [18]:
fasta_file = SeqIO.parse("dna2.fasta","fasta")

list_seq=[]
subseq_total = []
for seq_record in fasta_file:
    record_str = str(seq_record.seq)
    list_seq.append(record_str)
    print(seq_record)

    subseq_n = count_repeat_elements(record_str,10)
    
    for i in subseq_n:
        subseq_total.append(i)

ID: gi|142022655|gb|EQ086233.1|91
Name: gi|142022655|gb|EQ086233.1|91
Description: gi|142022655|gb|EQ086233.1|91 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
Number of features: 0
Seq('CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGG...GCC')
ID: gi|142022655|gb|EQ086233.1|304
Name: gi|142022655|gb|EQ086233.1|304
Description: gi|142022655|gb|EQ086233.1|304 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
Number of features: 0
Seq('CGCGATCCAGTAGCGCTTGTAGCCGAGCGCTTCGGCACGCTTCGCGAGCGCGAT...GCC')
ID: gi|142022655|gb|EQ086233.1|255
Name: gi|142022655|gb|EQ086233.1|255
Description: gi|142022655|gb|EQ086233.1|255 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
Number of features: 0
Seq('CTCGACGCGCTCCGCGTCGAGGTCGCCCGACGTCTCGCGCAGCAACTGATTCAA...TCG')
ID: gi|142022655|gb|EQ086233.1|45
Name: gi|142022655|gb|EQ086233.1|45
Description: gi|142022655|gb|EQ086233.1|45 ma

In [19]:
# Through Counter it will be possible to return the dictionary with the repetitions in the sequences (key) 
# ordered from highest to lowest by their frequency (value)

from collections import Counter
Counter(subseq_total).most_common()

[('CATTCGCCAT', 10),
 ('ATTCGCCATT', 10),
 ('TTCGCCATTC', 10),
 ('TCGCCATTCG', 10),
 ('CGCCATTCGC', 10),
 ('GCCATTCGCC', 10),
 ('CCATTCGCCA', 9),
 ('GCGCGCGGCG', 7),
 ('CGGCGTGTCG', 6),
 ('CAGCGCCGCG', 6),
 ('GCCGCGCTCG', 6),
 ('GCAGCGCGGC', 6),
 ('GTGCGCGCCG', 6),
 ('CGGCGCGGTG', 6),
 ('GCGATCGCGC', 6),
 ('GCCGATGCCG', 6),
 ('CGATCGTCGC', 6),
 ('GATCGTCGCG', 6),
 ('GCGGCCGCGC', 6),
 ('CGGCCGCGCG', 6),
 ('CGCGCGACGC', 6),
 ('GCGCGACGCG', 6),
 ('CGACGCTCGC', 6),
 ('GCGCTGGCCG', 6),
 ('GCGCTGCTGC', 6),
 ('CGCGGTCGAT', 6),
 ('CGTCGGCGAT', 5),
 ('CGCCGGCCGG', 5),
 ('GCCGGCCGGC', 5),
 ('CGCGATGCTG', 5),
 ('CGCCGCCGCG', 5),
 ('CAGCGCGGCG', 5),
 ('GCGCGGCGGC', 5),
 ('GCGGCGCTCG', 5),
 ('CGGCGTCCGG', 5),
 ('TGCTGCGCGC', 5),
 ('GCTGCGCGCG', 5),
 ('TGCGCGCCGA', 5),
 ('CTCGCGGCCG', 5),
 ('GCGCGGCATC', 5),
 ('CGCGCCGCGT', 5),
 ('CAGGTCGCGC', 5),
 ('CGTCGTGCGC', 5),
 ('GTCGTGCGCG', 5),
 ('TCGTGCGCGC', 5),
 ('CGACGTCGAG', 5),
 ('GCCGCGGCGC', 5),
 ('CTCGCGCAGC', 5),
 ('GGCGCGCGGC', 5),
 ('CGGCGATCGC'